# Assignment Scenario

Congratulations! You have just been hired by a US Venture Capital firm as a data analyst.

The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price volatility. For now they are mainly interested in a macro-view of Canada's crop farming industry, and how it relates to the relative value of the Canadian and US dollars.


# Introduction

Using this R notebook you will:

1.  Understand four datasets 
2.  Load the datasets into four separate tables in a Db2 database
3.  Execute SQL queries unsing the RODBC R package to answer assignment questions 

You have already encountered two of these datasets in the previous practice lab. You will be able to reuse much of the work you did there to prepare your database tables for executing SQL queries.


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of two datasets from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01&pid=3210007701">Farm product prices (Data & Metadata)</a>
3.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork890-2022-01-01">Bank of Canada daily average exchange rates</a>


### 1. Canadian Principal Crops Data *

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901  
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.  

### 2. Farm product prices

This dataset contains monthly average farm product prices for Canadian crops and livestock by province and teritory, from 1980 to 2020 (or 'last year', whichever is greatest).

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701 
The information is included in the metadata file, which can be downloaded - look for the 'download options' link.  

### 3. Bank of Canada daily average exchange rates *

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section). We have also prepared a monthly averaged version which you will be using below.

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/

( * these datasets are the same as the ones you used in the practice lab)


### Dataset URLs

  1.  Annual Crop Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv 

  2.  Farm product prices: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv
  
  3.  Daily FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv
  
  4.  Monthly FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv
  

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from these URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


#### Now let's load these datasets into four separate Db2 tables.
Let's first load the RODBC package:


In [ ]:
library(RODBC)

## Problem 1
#### Create tables
Establish a connection to the Db2 database, and create the following four tables using the RODBC package in R. 
Use the separate cells provided below to create each of your tables.

1.  **CROP_DATA**
2.  **FARM_PRICES**
3.  **DAILY_FX**
4.  **MONTHLY_FX**  

The previous practice lab will help you accomplish this.


### Solution 1


In [ ]:
# Establish database connection
    dsn_driver <- "{IBM DB2 ODBC Driver}"
    dsn_database <- "bludb"            # e.g. "bludb"
    dsn_hostname <- "b0aebb68-94fa-46ec-a1fc-1c999edb6187.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" # e.g "54a2f15b-5c0f-46df-8954-.databases.appdomain.cloud"
    dsn_port <- "31249"   # e.g. "32733" 
    dsn_protocol <- "TCPIP"            # i.e. "TCPIP"
    dsn_uid <- "bkv07106"        # e.g. "zjh17769"
    dsn_pwd <- "Ojka0A5SzsilXsVY"      # e.g. "zcwd4+8gbq9bm5k4"  
    dsn_security <- "ssl"

    conn_path <- paste("DRIVER=",dsn_driver,
                  ";DATABASE=",dsn_database,
                  ";HOSTNAME=",dsn_hostname,
                  ";PORT=",dsn_port,
                  ";PROTOCOL=",dsn_protocol,
                  ";UID=",dsn_uid,
                  ";PWD=",dsn_pwd,
                  ";SECURITY=",dsn_security,        
                    sep="")
    conn <- odbcDriverConnect(conn_path)

    # Dump connection info
    ##############################################################
    sql.info <- sqlTypeInfo(conn)
    conn.info <- odbcGetInfo(conn)
    conn.info["DBMS_Name"]
    conn.info["DBMS_Ver"]
    conn.info["Driver_ODBC_Ver"]

In [78]:
tables <- c("CROP_DATA", "DAILY_FX", "FARM_PRICES", "MONTHLY_FX") 

    for (table in tables) {
        # Drop tables if they already exist
        out <- sqlTables(conn, tableType = "TABLE",
                          tableName = table)
        if (nrow(out)>0) {
            err <- sqlDrop(conn, table,
                            errors=FALSE)  
            if (err==-1) {
                cat("An error has occurred.\n")
                err.msg <- odbcGetErrMsg(conn)
                for (error in err.msg) { 
                    cat(error,"\n")
                }
            } 
            else {
                cat ("Table: ",table," was dropped\n")
            }
        }
        else {
              cat ("Table: ", table," does not exist\n")
        }
    }

Table:  CROP_DATA  was dropped
Table:  DAILY_FX  was dropped
Table:  FARM_PRICES  was dropped
Table:  MONTHLY_FX  was dropped


In [79]:
# CROP_DATA:
 df1 <- sqlQuery(conn, 
                    "CREATE TABLE CROP_DATA (
                                      CD_ID INTEGER NOT NULL,
                                      YEAR DATE NOT NULL,
                                      CROP_TYPE VARCHAR(20) NOT NULL,
                                      GEO VARCHAR(20) NOT NULL, 
                                      SEEDED_AREA INTEGER NOT NULL,
                                      HARVESTED_AREA INTEGER NOT NULL,
                                      PRODUCTION INTEGER NOT NULL,
                                      AVG_YIELD INTEGER NOT NULL,
                                      PRIMARY KEY (CD_ID)
                                      )", 
                    errors=FALSE
                    )

    if (df1 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [80]:
# FARM_PRICES:
 df2 <- sqlQuery(conn, "CREATE TABLE FARM_PRICES (
                                CD_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                CROP_TYPE VARCHAR(20) NOT NULL,
                                GEO VARCHAR(20) NOT NULL, 
                                PRICE_PERMIT INTEGER NOT NULL
                                )",
                    errors=FALSE
                    )

    if (df2 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [81]:
# DAILY_FX:
    df3 <- sqlQuery(conn, "CREATE TABLE DAILY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df3 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


In [82]:
# MONTHLY_FX:
 df4 <- sqlQuery(conn, "CREATE TABLE MONTHLY_FX (
                                DFX_ID INTEGER NOT NULL,
                                DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (DFX_ID)
                                )",
                    errors=FALSE
                    )

    if (df4 == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

Table was created successfully.


## Problem 2
#### Read Datasets and Load Tables
Read the datasets into R dataframes using the urls provided above. Then load your tables.


###  Solution 2


In [83]:
crop_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv")
prices_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv") 
dfx_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv")
mfx_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv")

head(crop_df)
head(prices_df)
head(dfx_df)
head(mfx_df)

,CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<fct>,<fct>,<fct>,<dbl>
1,0,1985-01-01,Barley,Alberta,127.39
2,1,1985-01-01,Barley,Saskatchewan,121.38
3,2,1985-01-01,Canola,Alberta,342.00
4,3,1985-01-01,Canola,Saskatchewan,339.82
5,4,1985-01-01,Rye,Alberta,100.77
6,5,1985-01-01,Rye,Saskatchewan,109.75


,DFX_ID,DATE,FXUSDCAD
,<int>,<fct>,<dbl>
1,0,2017-01-03,1.3435
2,1,2017-01-04,1.3315
3,2,2017-01-05,1.3244
4,3,2017-01-06,1.3214
5,4,2017-01-09,1.3240
6,5,2017-01-10,1.3213


,DFX_ID,DATE,FXUSDCAD
,<int>,<fct>,<dbl>
1,0,2017-01-01,1.319276
2,1,2017-02-01,1.310726
3,2,2017-03-01,1.338643
4,3,2017-04-01,1.344021
5,4,2017-05-01,1.360705
6,5,2017-06-01,1.329805


In [ ]:
sqlSave(conn, crop_df, "CROP_DATA", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, prices_df, "FARM_PRICES", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, dfx_df, "DAILY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, mfx_df, "DAILY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)

In [ ]:
tryCatch(
  {
    sqlSave(conn, crop_df, "CROP_DATA", append = TRUE, fast = FALSE, rownames = FALSE, colnames = FALSE, verbose = FALSE)
  },
  error = function(e) {
    cat("Error message:", conditionMessage(e), "\n")
  }
)


## Now execute SQL queries using the RODBC R package to solve the assignment problems.

## Problem 3
#### How many records are in the farm prices dataset?


### Solution 3


In [ ]:
query = "SELECT COUNT(CD_ID) FROM FARM_PRICES"
    sqlQuery(conn,query)

## Problem 4
#### Which geographies are included in the farm prices dataset?


### Solution 4


In [ ]:
query <- "SELECT DISTINCT GEO FROM FARM_PRICES"
sqlQuery(conn, query)

## Problem 5
#### How many hectares of Rye were harvested in Canada in 1968?


### Solution 5


In [ ]:
query <- "SELECT HARVESTED_AREA  
          FROM CROP_DATA 
           WHERE YEAR = '1968' AND CROP_TYPE = 'Rye'"
sqlQuery(conn, query)

## Problem 6
#### Query and display the first 6 rows of the farm prices table for Rye.


### Solution 6


In [ ]:
query <- "SELECT * FROM FARM_PRICES WHERE CROP_TYPE='Rye' LIMIT 6"
    view <- sqlQuery(conn,query)
    view

## Problem 7
#### Which provinces grew Barley? 


### Solution 7


In [ ]:
query <- "SELECT DISTINCT GEO FROM CROP_DATA WHERE CROP_TYPE = 'Barley'" 
 sqlQuery(conn, query)

## Problem 8
#### Find the first and last dates for the farm prices data.


### Solution 8


In [ ]:
query <- "SELECT MIN(DATE), MAX(DATE) FROM FARM_PRICES"
sqlQuery(conn, query)

## Problem 9
#### Which crops have ever reached a farm price greater than or equal to &#0036;350 per metric tonne?


### Solution 9


In [ ]:
query <- "SELECT DISTINCT CROP_TYPE FROM FARM_PRICES WHERE PRICE_PRERMT > '350'"
sqlQuery(conn, query)

## Problem 10
#### Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best? 


### Solution 10


In [ ]:
query <- "SELECT CROP_TYPE, AVG(AVG_YIELD) AS avg_yield
         FROM CROP_DATA 
         WHERE YEAR = '2000' AND GEO = 'Saskatchewan'
         GROUP BY CROP_TYPE
         ORDER BY avg_yield DESC"
 <- sqlQuery(conn, query)


## Problem 11
#### Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000? 


### Solution 11


In [ ]:
query <- "SELECT GEO, CROP_TYPE, AVG(AVG_YIELD) AS avg_yield
         FROM CROP_DATA
         WHERE YEAR >= 2000  
         GROUP BY GEO, CROP_TYPE
         ORDER BY avg_yield DESC
         LIMIT 1"

<- sqlQuery(conn, query)

## Problem 12
#### Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.


### Solution 12


In [ ]:
query <- "SELECT HARVESTED_AREA
         FROM CROP_DATA
         WHERE YEAR = (SELECT MAX(YEAR) FROM CROP_DATA) AND  
               CROP_TYPE = 'Wheat' AND GEO = 'Canada'"

sqlQuery(conn, query)

## Problem 13
#### Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.


### Solution 13


In [ ]:
query <- "SELECT FP.DATE, FP.PRICE_PRERMT, MFX.FXUSDCAD,  
           FP.PRICE_PRERMT * MFX.FXUSDCAD AS PricePerTonCAD
         FROM FARM_PRICES FP
         INNER JOIN MONTHLY_FX MFX
           ON FP.DATE = MFX.DATE
         WHERE FP.GEO = 'Saskatchewan' AND FP.CROP_TYPE = 'Canola'  
         ORDER BY FP.DATE DESC
         LIMIT 6"

sqlQuery(conn, query)